## SET UP

In [3]:
#GET DATA FROM CASSANDRA AS DATAFRAME
from queries import make_queries_get_df

#FOR MANIPULATING DATAFRAME
import pandas as pd

#FOR MAKING QUICK CHARTS
import chart_helper

#PLOTTING (THIS IS OPTIONAL IF YOU WISH TO TWEAK A CHART FURTHER)
import plotly.express as px

## 1st Example: Total File Usage Across All Courses

The following query can be used to get an overview of file usage across different courses. If needed, the table can also add an extra layer of grouping such as department and university.

### Step 1: Get the Data from Cassandra

Notice that Python only transforms the result into a table and visualise it into a sunburst chart, as the data is already prepared by Cassandra.  

The actual query can be seen in the *string* argument passed onto the make_queries_get_df() function.

In [4]:
# get data for file usage

file_usage_df = make_queries_get_df('''
SELECT course_id, paper_id, document_id, type, SUM(size) as total_file_usage_in_KB
FROM component
GROUP BY course_id, paper_id, document_id, type;
''')

In [10]:
file_usage_in_mb = pd.DataFrame.copy(file_usage_df, deep=True)
file_usage_in_mb['file_usage_in_mb'] = round(file_usage_in_mb['total_file_usage_in_kb']/1000, 2)
file_usage_in_mb.drop(columns=['total_file_usage_in_kb'], inplace=True)

#the quick_sunburst() function works as long as the columns are in the right order
#something like: 1st layer > 2nd layer > ... > values_columns
#for the chart in question: course > paper > doc > type > size
file_usage_in_mb_fig = chart_helper.quick_sunburst(file_usage_in_mb)

file_usage_in_mb_fig.show()

file_usage_in_mb_fig.write_html('file_usage_in_mb.html', auto_open=True)

## 2nd example: getting contributions by each individuals in a paper

The below queries can be run to produce an instant breakdown of contributions by each student to each document in a paper. 

If an instructor's teaching multiple papers, similar queries can be made for each paper and they can be implemented as separate option from a drop-down menu.  

### Step 1: Get Data as Dataframe from Cassandra

The helper function *make_queries_get_df()* accepts a string of CQL statement as its argument.

It hides the details of connecting to Cassandra and executing CQL query using Python.

This means you can test out your CQL statement on Datastax Studio, and if you're happy with the result, you can simply copy and paste it into your Jupyter notebook.

The returned data will be a dataframe, ready to be processed and visualised.

In [12]:
contribution_by_paperC1_df = make_queries_get_df('''
SELECT paper_id, document_id, author_full_name, type, COUNT(type) as count
FROM component
WHERE course_id = 'courseC'
AND paper_id = 'paperC1'
GROUP BY document_id, type;
''')


Notice that the data's already sorted and grouped by Cassandra on the server side. This takes advantage of the design of the component table and does not force processing on the client side.

Similar to the 1st example, we use *quick_sunburst()* to visualise the received dataframe, and save it to an external html file.

In [13]:
contribution_by_paperC1_fig = chart_helper.quick_sunburst(contribution_by_paperC1_df)

contribution_by_paperC1_fig.write_html('contribution_by_paperC1_fig.html', auto_open=True)